# Path Visualization

In [ ]:
import os
import re
from math import isnan, nan
from typing import cast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.artist import Artist
from matplotlib.axes import Axes
from matplotlib.ticker import ScalarFormatter

## Data Loading

In [ ]:
def create_route():
    return {
        "__plot_handle": None,
        "Origin_x": nan,
        "Origin_y": nan,
        "Initial_dir": nan,
        "End_dir": nan,
        "Left_right_sign": nan,
        "PathNum": nan,
        "Path": [
            {
                "PathType": nan,
                "TraceType": nan,
                "EntryPoint_x": nan,
                "EntryPoint_y": nan,
                "ExitPoint_x": nan,
                "ExitPoint_y": nan,
                "CenterPoint_x": nan,
                "CenterPoint_y": nan,
                "R": nan,
                "X_data": [nan] * 10,
                "Y_data": [nan] * 10,
            }
            for _i in range(5)
        ],
    }


routes = {
    "front": create_route(),
    "rear": create_route(),
}

ROUTE_PATTERN = re.compile(
    r"""(?x)
        g(?P<route_type>Front|Rear)Route\.
        (?P<key>\w+)\ =\ (?P<value>-?\d*(?:\.\d*)?)
    """
)
PATH_PATTERN = re.compile(
    r"""(?x)
        g(?P<route_type>Front|Rear)Route\.
        Path\[(?P<path_index>\d+)\]\.
        (?P<key>\w+)\ =\ (?P<value>-?\d*(?:\.\d*)?)
    """
)
DATA_PATTERN = re.compile(
    r"""(?x)
        g(?P<route_type>Front|Rear)Route\.
        Path\[(?P<path_index>\d+)\]\.
        (?P<key>\w+)\[(?P<list_index>\d+)\]\ =\ (?P<value>-?\d*(?:\.\d*)?)
    """
)

candidate_log_files = [
    file_name
    for file_name in os.listdir(".")
    if "tracking_control" in file_name and file_name.endswith(".log")
]
if len(candidate_log_files) != 1:
    raise RuntimeError(
        f"Unexpected log file candidate(s): {candidate_log_files!r}"
    )

with open(candidate_log_files[0], "r") as log_file:
    for line in log_file:
        if "[ INFO]" not in line:
            continue

        line = line.strip()
        escape_seq_len = line.index("[ INFO]")
        if escape_seq_len > 0:
            line = line[escape_seq_len:-escape_seq_len]

        if match := ROUTE_PATTERN.search(line):
            route_type = match.group("route_type").lower()
            path_index = -1
            list_index = -1
            key = match.group("key")
            value = match.group("value")
        elif match := PATH_PATTERN.search(line):
            route_type = match.group("route_type").lower()
            path_index = int(match.group("path_index"))
            list_index = -1
            key = match.group("key")
            value = match.group("value")
        elif match := DATA_PATTERN.search(line):
            route_type = match.group("route_type").lower()
            path_index = int(match.group("path_index"))
            list_index = int(match.group("list_index"))
            key = match.group("key")
            value = match.group("value")
        else:
            continue

        if "." in value:
            value = float(value)
        else:
            value = int(value)

        route = routes[route_type]
        if path_index == -1:
            if key not in route:
                continue
            # ignore duplicate path attributes in log file
            route[key] = value
        else:
            path = route["Path"][path_index]
            if key not in path:
                continue
            if list_index == -1:
                if not isnan(path[key]):
                    break
                path[key] = value
            else:
                list_ = path[key]
                if not isnan(list_[list_index]):
                    break
                list_[list_index] = value

print(f"Loaded {candidate_log_files[0]!r}.")

In [ ]:
df_raw = pd.read_csv(
    "tracking_control_node.csv",
    index_col=0,
    parse_dates=["timestamp"],
)
df_plot = df_raw.query("valid == 1")

asc_flag = "x_vf" in df_plot.columns
if asc_flag:
    front_label, rear_label = "vf", "vr"
else:
    front_label, rear_label = "f", "r"

In [ ]:
if asc_flag:
    TRANS_MAP_DIR_SUFFIX = "agv_ns_ros/data/AGV_Map/"
else:
    TRANS_MAP_DIR_SUFFIX = "agv_ns_ros_agv/data/AGV_Map/"

CWD = os.getcwd()
current_dir = os.path.dirname(CWD)
while (parent_dir := os.path.dirname(current_dir)) != current_dir:
    current_dir = parent_dir
    map_dir_path = os.path.join(current_dir, TRANS_MAP_DIR_SUFFIX)
    if os.path.exists(map_dir_path):
        TRANS_MAP_DIR = map_dir_path
        break
else:
    raise FileNotFoundError("Failed to find the transponder map file!")

trans_map_candidates = [
    file_name
    for file_name in os.listdir(TRANS_MAP_DIR)
    if file_name.startswith("TransMap") and file_name.endswith(".csv")
]
if len(trans_map_candidates) < 1:
    raise FileNotFoundError("Transponder map file not found!")
else:
    selected_trans_map_file_name = sorted(trans_map_candidates)[-1]

TRANS_MAP_PATH = os.path.join(TRANS_MAP_DIR, selected_trans_map_file_name)
print(f'Selected "{os.path.relpath(TRANS_MAP_PATH, CWD)}".')
df_trans_map = pd.read_csv(
    TRANS_MAP_PATH,
    names=["TransID", "AbsX", "AbsY", "LaneNB1", "LaneNB2"],
    index_col=0,
)
df_trans_map["x"] = df_trans_map["AbsX"] / 1000
df_trans_map["y"] = df_trans_map["AbsY"] / 1000


def plot_transponders(ax: Axes) -> None | Artist:
    handle = None
    for id in df_trans_map.index:
        handle = ax.plot(
            cast(np.float64, df_trans_map.loc[id, "x"]),
            cast(np.float64, df_trans_map.loc[id, "y"]),
            "+",
            ms=5,
            color="purple",
            label="Transponders",
        )[0]
    return handle

## Entry/Exit Points

In [ ]:
first_front_path = routes["front"]["Path"][0]
first_path_dir = np.array([
    first_front_path["ExitPoint_x"] - first_front_path["EntryPoint_x"],
    first_front_path["ExitPoint_y"] - first_front_path["EntryPoint_y"],
])
initial_vehicle_dir = np.array([
    df_plot["x_" + front_label].iloc[0] - df_plot["x_" + rear_label].iloc[0],
    df_plot["y_" + front_label].iloc[0] - df_plot["y_" + rear_label].iloc[0],
])
if np.dot(first_path_dir, initial_vehicle_dir) > 0:
    target_axle = "front"
    route_label = front_label
else:
    target_axle = "rear"
    route_label = rear_label

route = routes[target_axle]
paths = route["Path"]

print(f"PathNum: {route['PathNum']:d}")
for i in range(route["PathNum"]):
    path = paths[i]
    print(
        "Path[%d]: (%7.3f, %7.3f) -> (%7.3f, %7.3f)" % (
            i,
            path["EntryPoint_x"],
            path["EntryPoint_y"],
            path["ExitPoint_x"],
            path["ExitPoint_y"],
        )
    )
print(
    "Final Position: (%7.3f, %7.3f)" % (
        df_plot[f"x_{route_label}"].iloc[-1],
        df_plot[f"y_{route_label}"].iloc[-1],
    )
)

fig = plt.figure(figsize=(6, 3), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

plot_slice = slice(-min(len(df_plot), 150), None)
last_path = paths[route["PathNum"] - 1]
target_axis = (
    "x" if last_path["ExitPoint_x"] != last_path["EntryPoint_x"] else "y"
)
target_label = f"{target_axis}_{route_label}"
v_plot = df_plot[target_label]
v_ref = last_path[f"ExitPoint_{target_axis}"]
ax.plot(
    df_plot["timestamp"].iloc[plot_slice],
    v_plot.iloc[plot_slice],
    "b.-",
    ms=2,
    alpha=0.3,
    label="Actual Position",
)

ax.axhline(v_ref, ls="--", color="gold", alpha=0.8, label="Target Position")

for label in ax.get_xticklabels():
    label.set_rotation(15)

ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))

ax.set(
    xlabel="Timestamp",
    ylabel=target_label,
)
ax.legend()
ax.grid()

## Visualization

In [ ]:
PLOT_PADDING = 2


def visualize(items: list[tuple[str, tuple[str, str]]]) -> None:
    """
    Args:
        items (list[tuple[str, tuple[str, str]]]):
            (route_type, (design_color, actual_color)), ...
    """

    fig = plt.figure(figsize=(5, 5), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    x_min = y_min = np.inf
    x_max = y_max = -np.inf

    for route_type, colors in items:
        route = routes[route_type]
        plot_kwargs = dict(
            lw=6,
            ms=12,
            color=colors[0],
            alpha=0.5,
            label=f"Design path ({route_type})",
        )
        # draw circle reference first
        for path in route["Path"][:route["PathNum"]]:
            if path["TraceType"] == 2:  # circle
                theta = np.linspace(0, 2 * np.pi, 100)
                x = path["R"] * np.cos(theta) + path["CenterPoint_x"]
                y = path["R"] * np.sin(theta) + path["CenterPoint_y"]
                ax.plot(x, y, "--", color="gray", alpha=0.5)  # type: ignore
                ax.plot(
                    path["CenterPoint_x"],
                    path["CenterPoint_y"],
                    "+",
                    color="gray",
                    alpha=0.5,
                )
                x_min, y_min = min(x_min, x.min()), min(y_min, y.min())
                x_max, y_max = max(x_max, x.max()), max(y_max, y.max())
        # draw other paths
        for path in route["Path"][:route["PathNum"]]:
            match path["TraceType"]:
                case 1:  # straight line
                    x = np.array([path["EntryPoint_x"], path["ExitPoint_x"]])
                    y = np.array([path["EntryPoint_y"], path["ExitPoint_y"]])
                    route["__plot_handle"] = ax.plot(
                        x,
                        y,
                        ".-",
                        **plot_kwargs,  # type: ignore
                    )[0]
                    x_min, y_min = min(x_min, x.min()), min(y_min, y.min())
                    x_max, y_max = max(x_max, x.max()), max(y_max, y.max())
                case 2:  # circle
                    pass
                case 3 | 4:  # arbitrary curve
                    x0 = route["Origin_x"]
                    y0 = route["Origin_y"]
                    x1 = np.array(path["X_data"])
                    y1 = np.array(path["Y_data"])
                    theta = route["Initial_dir"]
                    sign = route["Left_right_sign"]
                    x = x1 * np.cos(theta) - sign * y1 * np.sin(theta) + x0
                    y = x1 * np.sin(theta) + sign * y1 * np.cos(theta) + y0
                    route["__plot_handle"] = ax.plot(
                        x,
                        y,
                        ".-",
                        **plot_kwargs,  # type: ignore
                    )[0]
                    x_min, y_min = min(x_min, x.min()), min(y_min, y.min())
                    x_max, y_max = max(x_max, x.max()), max(y_max, y.max())
                case _ as v:
                    raise ValueError(f"unexpected trace type: {v!r}")

    handles_actual = []
    for route_type, colors in items:
        route_label = front_label if route_type == "front" else rear_label
        x = df_plot["x_" + route_label]
        y = df_plot["y_" + route_label]
        handles_actual.append(
            ax.plot(
                df_plot["x_" + route_label],
                df_plot["y_" + route_label],
                ".-",
                ms=2,
                color=colors[1],
                alpha=0.3,
                label=f"Actual path ({route_type})",
            )[0]
        )
        x_min, y_min = min(x_min, x.min()), min(y_min, y.min())
        x_max, y_max = max(x_max, x.max()), max(y_max, y.max())

    handle_transponders = plot_transponders(ax)

    x_mid = (x_min + x_max) / 2
    y_mid = (y_min + y_max) / 2
    radius = max(x_max - x_mid, y_max - y_mid)
    x_min, x_max = x_mid - radius, x_mid + radius
    y_min, y_max = y_mid - radius, y_mid + radius

    ax.set(
        xlabel="x",
        ylabel="y",
        xlim=(x_min - PLOT_PADDING, x_max + PLOT_PADDING),
        ylim=(y_min - PLOT_PADDING, y_max + PLOT_PADDING),
        aspect="equal",
    )
    ax.legend(
        handles=(
            *[routes[route_type]["__plot_handle"] for route_type, _ in items],
            *handles_actual,
            handle_transponders,
        ),
    )
    ax.grid()

### Front & Rear

In [ ]:
visualize([
    ("front", ("red", "pink")),
    ("rear", ("blue", "cyan")),
])

### Front Only

In [ ]:
visualize([
    ("front", ("red", "pink")),
])

### Rear Only

In [ ]:
visualize([
    ("rear", ("blue", "cyan")),
])